# Libraries

In [ ]:
import os
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
from rasterio.enums import Resampling
from datetime import datetime, timedelta
from calendar import monthrange

In [ ]:
# Set the base directory with an environment variable, fallback to a relative path
main_dir = os.getenv('PROJECT_DIR', os.path.join(os.getcwd(), 'data', 'ANIN', 'Generating Indices', 'VCI', 'NDVI-16', 'Trial', 'Tiles', 'Crop_mask_new'))

main_dir = r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\VCI\NDVI-16\Trial\Tiles\Crop mask\Resampled to 300m'
main_dir = r'Insert the path to the directory with the resampled to 300m data'
Base_crop_mask = rxr.open_rasterio(main_dir  +'\\' + 'Base_crop_mask300.tif',masked=True)
Crop_layer = rxr.open_rasterio(main_dir+ '\\' + 'Cropland300.tif',masked=True)
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
          'October', 'November', 'December']

In [ ]:
Phenology = Base_crop_mask.drop(labels = 1, dim = 'band')
Phenology_new = xr.where((Phenology > 36) & (Phenology <= 72), Phenology - 36, Phenology)
Phenology_new = xr.where(Phenology_new> 72 , Phenology_new - 72,Phenology_new)

In [ ]:
count = 0
Phenology_month = Phenology_new
for i in range(0,12):
    Phenology_month = xr.where((Phenology_new > (0 + count)) & (Phenology_new <= (3 +count)), 1+i, Phenology_month)
    count = count+3
    

In [ ]:
m = xr.concat([Crop_layer,Phenology_month], dim = 'band')

In [ ]:
for month in range(0,12):
    Crop_mask = xr.where( (m[0].isnull() ==False) & ((m[1]==month+1) | (m[2] ==month+1) | ((m[1]>month+1) & (m[2]<m[1]) & (m[2]>month+1)) | ((m[1]<month+1) & (m[2]>month+1))), 1,0)
    Crop_mask = Crop_mask.where(Crop_mask!=0)
    
    Crop_mask.rio.to_raster(main_dir+'\\'+'Months'+ '\\' + 'Crop'+'\\'+ months[month]+ '.tif')